In [8]:
import pandas as pd
from langdetect import detect
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import string
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
df = pd.read_csv('../data/processedDict.csv')

In [10]:
lemmatised_words = []
df['word'] = df['word'].apply(lambda x: lem.lemmatize(x, pos='n'))
pattern = '|'.join(df['word']) #augment dictionary to pattern form -> used for vectorised function application

In [11]:
filepath = '../data/en_reviews/london_en.csv'
reviews = pd.read_csv(filepath)
del reviews['Unnamed: 0']
comments_original = reviews['comments'] #to add back before saving to csv
reviews

,listing_id,id,date,reviewer_id,reviewer_name,comments,langs
0,11551,30672,2010-03-21,93896,Shar-Lyn,"The flat was bright, comfortable and clean and...",en
1,11551,32236,2010-03-29,97890,Zane,We stayed with Adriano and Valerio for a week ...,en
2,90700,337227,2011-06-27,311071,Miqua,it was all in all the perfect week!\r\nchilton...,en
3,90700,378738,2011-07-17,224367,Prateek,"I'll start with the host, and then move on to ...",en
4,90700,543840,2011-09-18,1115024,Jennifer,Great location. Plenty to do just steps outsid...,en
...,...,...,...,...,...,...,...
871,38605,261935383,2018-05-07,22636994,Shirley,"Great location , great host. Elisa & Dom provi...",en
872,38605,262591290,2018-05-09,21123920,Yirong,Great location and very comfortable room,en
873,90700,171592623,2017-07-18,11567369,Helen,Lovely apartment perfect for one or two people...,en
874,17402,276675678,2018-06-14,4599097,Darius,Newly refurbished 3 bed in central London - ro...,en


In [12]:
nltk.download('wordnet') #download corpus for lemmatizer to compare results with

[nltk_data] Downloading package wordnet to /Users/hiru/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
lem = WordNetLemmatizer() #create lemmatizer

In [14]:
table = str.maketrans('', '', string.punctuation) #mapping to strip punctuation in review

#strip punct of each review -> lemmatise -> output is list of words so join into sentences
lem_comments = reviews.comments.apply(lambda review: ' '.join(map(str, [lem.lemmatize(word.translate(table), pos='n') for word in review.lower().split()])))

count = lem_comments.str.count(pattern) # # of dictionary words found in each review
length = lem_comments.str.len() # review length

reviews['dict_proportion'] = count/length # save proportions

In [15]:
"Mean dictionary presence: {:.4%}".format(np.mean(reviews['dict_proportion']))

'Mean dictionary presence: 5.3328%'

In [16]:
reviews

,listing_id,id,date,reviewer_id,reviewer_name,comments,langs,dict_proportion
0,11551,30672,2010-03-21,93896,Shar-Lyn,"The flat was bright, comfortable and clean and...",en,0.079545
1,11551,32236,2010-03-29,97890,Zane,We stayed with Adriano and Valerio for a week ...,en,0.045643
2,90700,337227,2011-06-27,311071,Miqua,it was all in all the perfect week!\r\nchilton...,en,0.065315
3,90700,378738,2011-07-17,224367,Prateek,"I'll start with the host, and then move on to ...",en,0.058027
4,90700,543840,2011-09-18,1115024,Jennifer,Great location. Plenty to do just steps outsid...,en,0.058315
...,...,...,...,...,...,...,...,...
871,38605,261935383,2018-05-07,22636994,Shirley,"Great location , great host. Elisa & Dom provi...",en,0.033613
872,38605,262591290,2018-05-09,21123920,Yirong,Great location and very comfortable room,en,0.075000
873,90700,171592623,2017-07-18,11567369,Helen,Lovely apartment perfect for one or two people...,en,0.046025
874,17402,276675678,2018-06-14,4599097,Darius,Newly refurbished 3 bed in central London - ro...,en,0.050898


In [17]:
#overwrite csv in en_reviews
reviews.to_csv(filepath, index = False)